<a href="https://colab.research.google.com/github/veerendra12/CS598-DL4H-Project/blob/main/notebooks/Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings

In [ ]:
import humanize, psutil, GPUtil
import torch

import import_ipynb
from Configuration import CONFIG

In [ ]:
def install_libraries():
  !pip install -U albumentations  
  !pip uninstall opencv-python-headless --yes
  !pip install opencv-python-headless==4.5.2.52
  !pip list | grep opencv
  !pip install gputil
  !pip install psutil
  !pip install humanize

In [ ]:
def get_device():
  return "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def copy_results_from_drive():
  print('Copying data from drive to current working directory')

  base_dir = CONFIG['BASE_DIR']
  seg_base_dir = CONFIG['SEGMENTATION_BASE_DIR']
  seg_best_model_remote = CONFIG['SEGMENTATION_BEST_MODEL_DRIVE']

  !mkdir -p {seg_base_dir + 'results/'}
  !cp -v $seg_best_model_remote  {seg_base_dir + 'results/'} 

  !mkdir -p {base_dir + 'results/'}

  fem1_best_model_remote = CONFIG['FEM1_BEST_MODEL_DRIVE']
  fem1_best_model = CONFIG['FEM1_BEST_MODEL']
  !cp -v $fem1_best_model_remote $fem1_best_model

  fem2_best_model_remote = CONFIG['FEM2_BEST_MODEL_DRIVE']
  fem2_best_model = CONFIG['FEM2_BEST_MODEL']
  !cp -v $fem2_best_model_remote $fem2_best_model

  sdfn_best_model_remote = CONFIG['SDFN_BEST_MODEL_DRIVE']
  sdfn_best_model = CONFIG['SDFN_BEST_MODEL']
  !cp -v $sdfn_best_model_remote $sdfn_best_model

  best_results_remote = CONFIG['NIH_RESULTS_DRIVE_DIR']

  !cp -v {best_results_remote + '*true.csv'} {base_dir + 'results/'}
  !cp -v {best_results_remote + '*preds.csv'} {base_dir + 'results/'}
  !cp -v {best_results_remote + '*aucs.csv'} {base_dir + 'results/'}




In [ ]:
def copy_data_from_drive():
  print('Copying data from drive to current working directory')

  base_dir = CONFIG['BASE_DIR']
  seg_base_dir = CONFIG['SEGMENTATION_BASE_DIR']
  
  !mkdir -p {seg_base_dir + 'CXR_png/'}
  !mkdir -p {seg_base_dir + 'masks/'}
  !mkdir -p {seg_base_dir + 'results/saved_images/'}

  !mkdir -p $base_dir
  !mkdir -p {base_dir + 'results/saved_images'}

  image_zips = ['images_001.tar.gz', 
              'images_002.tar.gz', 
              'images_003.tar.gz',
              'images_004.tar.gz',
              'images_005.tar.gz',
              'images_006.tar.gz',
              'images_007.tar.gz',
              'images_008.tar.gz',
              'images_009.tar.gz',
              'images_010.tar.gz',
              'images_011.tar.gz',
              'images_012.tar.gz']
  
  base_dir = CONFIG['BASE_DIR']
  
  for image_zip in image_zips:
    remote_image_zip_path = CONFIG['NIH_DATASET_DRIVE_DIR'] + '/images/' + image_zip
    !cp $remote_image_zip_path $base_dir

    local_image_zip_path = CONFIG['NIH_DATASET_DRIVE_DIR']
    !cp /content/drive/MyDrive/Colab\ Notebooks/DataSets/NIHDataSet/*.csv $base_dir

    !tar -xf  {base_dir + image_zip} -C $base_dir

  copy_results_from_drive()

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

def memory_report():
  device = get_device()
  memory = {}

  memory['cpu-ram-free'] = humanize.naturalsize( psutil.virtual_memory().available)

  if device == 'cuda':
    memory['cuda-memory-allocated'] = humanize.naturalsize( torch.cuda.memory_allocated(0))
    memory['cuda-memory-cached'] = humanize.naturalsize( torch.cuda.memory_cached(0))

    GPUs = GPUtil.getGPUs()
    for i, gpu in enumerate(GPUs):
      memory['gpu-' + str(i) + '-mem-free'] = humanize.naturalsize(gpu.memoryFree * 1024 * 1024)
      memory['gpu-' + str(i) + '-mem-total'] = humanize.naturalsize(gpu.memoryTotal * 1024 * 1024)
      memory['gpu-' + str(i) + '-mem-util'] = str(round(gpu.memoryUtil, 3)) + '%'
      print(gpu.memoryFree)
  else:
    memory['cuda-memory-allocated'] = 0
    memory['cuda-memory-cached'] = 0
    memory['gpu-0-mem-free'] = 0
    memory['gpu-0-mem-total'] = 0
    memory['gpu-0-mem-util'] = 0

  return memory

In [ ]:
def load_checkpoint(model, filename):
    print("=> Loading checkpoint " + filename)
    if get_device() == 'cpu':
      model.load_state_dict(torch.load(filename, map_location='cpu')["state_dict"])
    else:
      model.load_state_dict(torch.load(filename)["state_dict"])

def save_checkpoint(state, filename):
    #print("=> Saving checkpoint " + filename)
    torch.save(state, filename)